<a href="https://colab.research.google.com/github/stygian2a/RoombaGym/blob/master/Roomba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO


In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [2]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!apt-get install x11-utils > /dev/null 2>&1
!wget https://raw.githubusercontent.com/stygian2a/RoombaGym/master/rendering.py

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (45.2.0)
--2020-02-09 13:46:32--  https://raw.githubusercontent.com/stygian2a/RoombaGym/master/rendering.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11819 (12K) [text/plain]
Saving to: ‘rendering.py’

rendering.py        100%[===================>]  11.54K  --.-KB/s    in 0s      

2020-02-09 13:46:32 (150 MB/s) - ‘rendering.py’ saved [11819/11819]



In [3]:
import gym
from gym import spaces
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython.display import clear_output
from time import sleep


from IPython import display as ipythondisplay

In [4]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:

class Roomba():
  def __init__(self, pos, direction, radius, battery, turn_angle=np.pi/18):
    self.pos = np.array(pos)
    self.direction = direction
    self.radius = radius
    self.turn_angle = turn_angle
    self.battery = battery
    self.max_battery = battery

  def set_pos(self, pos):
    self.pos = np.array(pos)
    self.battery -=1

  def get_forward(self):
    new_x = np.sin(self.direction) + self.pos[0]
    new_y = np.cos(self.direction) + self.pos[1]
    return [new_x, new_y]

  def get_backward(self):
    new_x = np.sin(self.direction+np.pi) + self.pos[0]
    new_y = np.cos(self.direction+np.pi) + self.pos[1]
    return [new_x, new_y]

  def turn_right(self):
    self.direction = (self.direction - self.turn_angle) % (2*np.pi)
    self.battery -=1


  def turn_left(self):
    self.direction = (self.direction + self.turn_angle) % (2*np.pi)
    self.battery -=1

  def recharge_battery(self):
    self.battery = self.max_battery

  def get_life(self):
    return self.battery / self.max_battery


  #https://stackoverflow.com/questions/49551440/python-all-points-on-circle-given-radius-and-center
  def get_cases_underneath(self, xmax, ymax, pos = None):
    if pos == None:
      pos = self.pos
    radius = self.radius
    x0 = pos[0] - 0.5
    y0 = pos[1] - 0.5

    L = []
    x_ = np.arange(x0 - radius - 1, x0 + radius + 1, dtype=int)
    y_ = np.arange(y0 - radius - 1, y0 + radius + 1, dtype=int)
    x, y = np.where((x_[:,np.newaxis] - x0)**2 + (y_ - y0)**2 <= radius**2)
    for x, y in zip(x_[x], y_[y]):
      if x>=0 and y>=0 and x<xmax and y<ymax:
        L.append((x, y))
    return L

In [0]:
class RoombaEnv(gym.Env):

  metadata = {
      'render.modes': ['human', 'rgb_array'],
      'video.frames_per_second' : 50
  }

  def __init__(self, height, width, battery, roomba_radius=2):
    super(RoombaEnv, self).__init__()

    self.action_space = spaces.Discrete(4) #Forward, Backwards, Right, Left
                                          
    self.observation_space = spaces.Tuple([
        spaces.Box(low=0, high=4, shape=(height, width), dtype=np.uint8), #0=empty/clean, 1=dirty, 2=wall, 3=charging station
        spaces.Box(low=np.array([0, 0, 0, 0]), high=np.array([height, width, 2*np.pi, 1]), dtype=np.float32)
    ])
    
    
    self.room = np.ones((height, width))
    obstacles = [[5,5], [5,6], [5,7], [5,8], [6,5], [7,5]]
    for obstIndex in obstacles:
      self.room[obstIndex[0],obstIndex[1]]= 2
    self.room[height-1, width-1] = 3
    self.roomba = Roomba( [height/2,width/2], 0, roomba_radius, battery)
    self.width = width
    self.height = height
    self.viewer = None
    self.done = False

    self.iteration = 0
    self.sum_rewards = 0

  def clean_floor(self):
    cases_underneath = self.roomba.get_cases_underneath(self.height, self.width)
    #TODO: optimize
    reward = 0
    for case_underneath in cases_underneath:
      if self.room[case_underneath] == 1:
        self.room[case_underneath] = 0
        reward += 1
      if self.room[case_underneath] == 0:
        reward -= 0
      if self.room[case_underneath] == 3:
        self.roomba.recharge_battery()
    return reward

  def step(self, action):
    self.iteration+=1
    
    if action == 0:
      new_pos = self.roomba.get_forward()
      if self.is_safe(new_pos):
        self.roomba.set_pos(new_pos)

    elif action == 1:
      new_pos = self.roomba.get_backward()
      if self.is_safe(new_pos):
        self.roomba.set_pos(new_pos)

    elif action == 2:
      self.roomba.turn_right()

    elif action == 3:
      self.roomba.turn_left()
      
    current_reward = self.clean_floor()

    if np.count_nonzero(self.room == 1) == 0:
      self.done = True
      current_reward += 1000

    elif self.roomba.battery <= 0 :
      current_reward += -1000

    self.sum_rewards += current_reward

    if self.iteration>=1000 or self.roomba.battery <= 0:
      self.done = True
      
    return self.get_observation(),current_reward,self.done,1

  def is_safe(self, new_pos):
    x = new_pos[0]
    y = new_pos[1]

    if y < 0 or x < 0 or x >= self.height or y >= self.width:
      return False

    cases_underneath = self.roomba.get_cases_underneath(self.height, self.width, new_pos)
    for case_underneath in cases_underneath:
      if self.room[case_underneath] == 2:
        return False

    return True

  def get_observation(self):
    return (self.room, np.array([self.roomba.pos[0], self.roomba.pos[1], self.roomba.direction, self.roomba.get_life()]))

  def reset(self):
    return self.get_observation()

  def render(self, mode='human', close=False):
    scale = 6

    screen_width = scale*self.width
    screen_height = scale*self.height


    if self.viewer is None:
      #TODO: remove when it works
      import importlib
      import rendering
      importlib.reload(rendering)

      self.viewer = rendering.Viewer(screen_width, screen_height)

      self.cases = []
      for i in range(self.height):
        self.cases.append([])
        for j in range(self.width):
          x0 = i*scale
          y0 = j*scale
          x1 = (i+1)*scale
          y1 = (j+1)*scale
          case = rendering.FilledPolygon([(y0,x0), (y0,x1), (y1,x1), (y1,x0)])
          self.cases[i].append(case)
          self.viewer.add_geom(case)
      
      circle = rendering.make_circle(radius=self.roomba.radius*scale)
      circle.set_color(1,0,0)
      direction_circle = rendering.make_circle(radius=self.roomba.radius*scale/2) #color?
      direction_circle.add_attr(rendering.Transform(translation=(self.roomba.radius*scale/2,0)))

      tracker = rendering.Compound([circle, direction_circle])

      self.tracker_trans = rendering.Transform()
      tracker.add_attr(self.tracker_trans)
      self.viewer.add_geom(tracker)

      
    for i in range(self.height):
        for j in range(self.width):
          if self.room[i,j] == 0:
            self.cases[i][j].set_color(1,1,1)
          elif self.room[i,j] == 1:
            self.cases[i][j].set_color(0.8,0.8,0.8)
          elif self.room[i,j] == 2:
            self.cases[i][j].set_color(0,0,0)
          elif self.room[i,j] == 3:
            self.cases[i][j].set_color(0.5,0.5,0)


    self.tracker_trans.set_translation(self.roomba.pos[1]*scale, self.roomba.pos[0]*scale)
    self.tracker_trans.set_rotation(self.roomba.direction)


    return self.viewer.render(return_rgb_array = mode=='rgb_array')

In [54]:
#env = wrap_env(RoombaEnv(10, 10, 2))
env = wrap_env(RoombaEnv(20, 20, 500, 2)) #wrap env seems to grow exponentially with H,W?? Maybe scalse
#env = wrap_env(gym.make("CartPole-v0"))
observation = env.reset()

while True:
  
    env.render()
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action)
    
    if done:
      break;
            
env.close()
show_video()

In [0]:
RoombaEnv(20, 20, 500, 2)
observation = env.reset()

while True:
  
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action)
    
    if done:
      break;
            
env.close()
